# Calculate additional performance metrics: Accuracy, balanced accuracy, sensitivity, specificity

In [1]:
import sys

sys.path.insert(0, '/home/sebastian/workspace/abcd_paper')

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import roc_curve, balanced_accuracy_score, accuracy_score

from src.definitions import PROCESSED_DATA_DIR, RESULTS_DIR

In [3]:
abcd_data = pd.read_csv(PROCESSED_DATA_DIR / 'abcd_data.csv', index_col='src_subject_id')

In [4]:
def get_scores(segmentation: str):
    bacc_sens_spec = {}

    for fold_i in range(150):
        filepath = Path(
            "/home/sebastian/workspace/abcd_paper/"
            f"work/{segmentation}_{fold_i:04d}_xgboost_cce/results/"
            f"run_unpermuted_seed77n30k5_{segmentation}_adjusted/"
            f"unpermuted/xgboost_cce/test/"
        )
        predictions = pd.read_csv(filepath / f"fold_{fold_i}.csv", index_col='src_subject_id')

        for diagnosis, predictions in predictions.items():
            y_true, y_pred = abcd_data[diagnosis].align(other=predictions, join='inner')
            fpr, tpr, threshold = roc_curve(
                y_true=y_true,
                y_score=y_pred
            )
            # Compute optimal threshold value, corresponding to maximum
            # Youden index
            optimal_idx = np.argmax(tpr - fpr)
            optimal_threshold = threshold[optimal_idx]
            # Compute sensitivity and specificity
            sensitivity = tpr[optimal_idx]
            specificity = 1 - fpr[optimal_idx]
            # Apply threshold to predictions and calculate balanced accuracy
            binary_predictions = predictions >= optimal_threshold
            balanced_accuracy = balanced_accuracy_score(
                y_true=y_true,
                y_pred=binary_predictions
            )
            accuracy = accuracy_score(
                y_true=y_true,
                y_pred=binary_predictions
            )
            # Store values
            if diagnosis not in bacc_sens_spec:
                bacc_sens_spec[diagnosis] = {
                    'accuracy': [], 'balanced_acc': [], 'sensitivity': [], 'specificity': []
                }
            bacc_sens_spec[diagnosis]['accuracy'].append(accuracy)
            bacc_sens_spec[diagnosis]['balanced_acc'].append(balanced_accuracy)
            bacc_sens_spec[diagnosis]['sensitivity'].append(sensitivity)
            bacc_sens_spec[diagnosis]['specificity'].append(specificity)

    return bacc_sens_spec


def aggregate_scores(bacc_sens_spec):
    scores = []

    for label, metrics in bacc_sens_spec.items():
        metrics = pd.DataFrame(metrics)
        df = pd.DataFrame({
            "mean": metrics.mean(),
            "min": metrics.min(),
            "max": metrics.max(),
        })
        df.columns = pd.MultiIndex.from_product(
            [(label,), df.columns]
        )

        scores.append(df)

    scores = pd.concat(scores, axis=1).T
    return scores

## FreeSurfer

In [5]:
aggregate_scores(get_scores("freesurfer")).round(3)

accuracy  balanced_acc  sensitivity  \
Major Depressive Disorder     mean     0.599         0.559        0.515   
                              min      0.152         0.508        0.034   
                              max      0.947         0.605        0.966   
Bipolar Disorder              mean     0.594         0.562        0.526   
                              min      0.233         0.515        0.138   
                              max      0.882         0.610        0.931   
Psychotic Symptoms            mean     0.468         0.556        0.649   
                              min      0.046         0.509        0.029   
                              max      0.976         0.630        1.000   
ADHD                          mean     0.552         0.561        0.577   
                              min      0.331         0.533        0.307   
                              max      0.714         0.597        0.893   
Oppositional Defiant Disorder mean     0.516         0.541        0.577   
                              min      0.209         0.510        0.136   
                              max      0.804         0.574        0.960   
Conduct Disorder              mean     0.583         0.574        0.566   
                              min      0.068         0.512        0.121   
                              max      0.945         0.671        1.000   
PTSD                          mean     0.584         0.559        0.531   
                              min      0.038         0.505        0.032   
                              max      0.977         0.622        1.000   
Obsessive Compulsive Disorder mean     0.498         0.531        0.572   
                              min      0.111         0.504        0.008   
                              max      0.910         0.577        1.000   
Generalized Anxiety Disorder  mean     0.545         0.536        0.527   
                              min      0.072         0.508        0.055   
                              max      0.936         0.588        1.000   
Social Anxiety Disorder       mean     0.542         0.552        0.563   
                              min      0.067         0.504        0.081   
                              max      0.928         0.611        1.000   

                                    specificity  
Major Depressive Disorder     mean        0.603  
                              min         0.116  
                              max         0.988  
Bipolar Disorder              mean        0.599  
                              min         0.178  
                              max         0.936  
Psychotic Symptoms            mean        0.463  
                              min         0.018  
                              max         1.000  
ADHD                          mean        0.546  
                              min         0.200  
                              max         0.803  
Oppositional Defiant Disorder mean        0.505  
                              min         0.080  
                              max         0.921  
Conduct Disorder              mean        0.583  
                              min         0.045  
                              max         0.965  
PTSD                          mean        0.586  
                              min         0.018  
                              max         0.999  
Obsessive Compulsive Disorder mean        0.491  
                              min         0.018  
                              max         0.999  
Generalized Anxiety Disorder  mean        0.546  
                              min         0.017  
                              max         0.985  
Social Anxiety Disorder       mean        0.541  
                              min         0.015  
                              max         0.975

## SRI24

In [6]:
aggregate_scores(get_scores("sri")).round(3)

accuracy  balanced_acc  sensitivity  \
Major Depressive Disorder     mean     0.492         0.548        0.609   
                              min      0.057         0.506        0.047   
                              max      0.933         0.599        1.000   
Bipolar Disorder              mean     0.527         0.545        0.567   
                              min      0.127         0.508        0.032   
                              max      0.928         0.599        0.980   
Psychotic Symptoms            mean     0.509         0.545        0.583   
                              min      0.046         0.507        0.025   
                              max      0.970         0.628        1.000   
ADHD                          mean     0.549         0.533        0.508   
                              min      0.265         0.508        0.059   
                              max      0.803         0.560        0.920   
Oppositional Defiant Disorder mean     0.537         0.537        0.536   
                              min      0.200         0.502        0.005   
                              max      0.854         0.570        0.947   
Conduct Disorder              mean     0.490         0.556        0.627   
                              min      0.025         0.500        0.077   
                              max      0.940         0.622        1.000   
PTSD                          mean     0.511         0.566        0.624   
                              min      0.023         0.500        0.032   
                              max      0.971         0.674        1.000   
Obsessive Compulsive Disorder mean     0.598         0.541        0.472   
                              min      0.134         0.506        0.069   
                              max      0.896         0.587        0.977   
Generalized Anxiety Disorder  mean     0.572         0.533        0.490   
                              min      0.059         0.502        0.027   
                              max      0.945         0.605        1.000   
Social Anxiety Disorder       mean     0.478         0.549        0.630   
                              min      0.098         0.510        0.079   
                              max      0.908         0.627        0.987   

                                    specificity  
Major Depressive Disorder     mean        0.486  
                              min         0.011  
                              max         0.976  
Bipolar Disorder              mean        0.524  
                              min         0.065  
                              max         0.994  
Psychotic Symptoms            mean        0.507  
                              min         0.018  
                              max         0.996  
ADHD                          mean        0.559  
                              min         0.112  
                              max         0.972  
Oppositional Defiant Disorder mean        0.537  
                              min         0.074  
                              max         0.999  
Conduct Disorder              mean        0.486  
                              min         0.001  
                              max         0.965  
PTSD                          mean        0.509  
                              min         0.001  
                              max         0.993  
Obsessive Compulsive Disorder mean        0.610  
                              min         0.048  
                              max         0.976  
Generalized Anxiety Disorder  mean        0.577  
                              min         0.004  
                              max         0.997  
Social Anxiety Disorder       mean        0.469  
                              min         0.048  
                              max         0.956